In [1]:
from tqdm import tqdm
from sklearn.decomposition import NMF
import pandas as pd
from scipy import sparse
import numpy as np

# read data
movies = pd.read_csv("data/ml-20m/movies.csv")
ratings = pd.read_csv("data/ml-20m/ratings.csv")

# join
ratings_joined = pd.merge(ratings, movies)

# ratingsをsparse matrixに変換して横持ちにする
action_adventure_ratings = ratings_joined.query("genres.str.contains('Action') or genres.str.contains('Adventure')", 
                                                engine='python').reset_index(drop=True)
# indexing ids
# userid
userid_unique = pd.Series(action_adventure_ratings["userId"].unique())
index_userid_dict = userid_unique.to_dict()
# inverse
userid_index_dict = dict(map(reversed, index_userid_dict.items()))

# itemid
itemid_unique = pd.Series(action_adventure_ratings["movieId"].unique())
index_itemid_dict = itemid_unique.to_dict()
# inverse
itemid_index_dict = dict(map(reversed, index_itemid_dict.items()))

action_adventure_ratings["user_id"] = action_adventure_ratings["userId"].map(userid_index_dict)
action_adventure_ratings["item_id"] = action_adventure_ratings["movieId"].map(itemid_index_dict)

# reindexしたidを使って、アイテムとジャンルの対応が取れるdictを作る
itemid_genres_dict = action_adventure_ratings[['item_id', 'genres']].set_index('item_id')['genres'].to_dict()

In [2]:
import cloudpickle
X_train = cloudpickle.load(open("output/ML-20M-X_train.pkl","rb"))
X_test = cloudpickle.load(open("output/ML-20M-X_test.pkl","rb"))

In [3]:
# aggregateのtrainをactionとadventureに分離する
# actionの列
action_columns = [v for v in range(X_train.shape[1]) if 'Action' in itemid_genres_dict[v]]
# adventureの列
adventure_columns = [v for v in range(X_train.shape[1]) if 'Adventure' in itemid_genres_dict[v]]

# 選んだカラムに応じてとってくる
action_train = X_train[:, action_columns]
adventure_train = X_train[:, adventure_columns]

# adventureのみ、アイテムidのconcatとの対応関係が必要なので辞書として持っておく
adventure_concat_itemid_dict = {}
count = 0
for v in range(X_train.shape[1]):
    if 'Adventure' in itemid_genres_dict[v]:
        adventure_concat_itemid_dict[v] = count
        count += 1

In [4]:
# アイテムidのconcatとの対応関係が必要なので辞書として持っておく
action_concat_itemid_dict = {}
count = 0
for v in range(X_train.shape[1]):
    if 'Action' in itemid_genres_dict[v]:
        action_concat_itemid_dict[v] = count
        count += 1
# inverse
inverse_action_concat_itemid_dict = dict(map(reversed, action_concat_itemid_dict.items()))

adventure_concat_itemid_dict = {}
count = 0
for v in range(X_train.shape[1]):
    if 'Adventure' in itemid_genres_dict[v]:
        adventure_concat_itemid_dict[v] = count
        count += 1
# inverse
inverse_adventure_concat_itemid_dict = dict(map(reversed, adventure_concat_itemid_dict.items()))

In [5]:
# それぞれにアクションしていないユーザを削る
# 全ユーザと、削ったあとでの対応関係を辞書として持っておく
action_train_selected = action_train[action_train.getnnz(1)>0]
adventure_train_selected = adventure_train[adventure_train.getnnz(1)>0]

action_train_action_users = {}
action_users = action_train.getnnz(1)>0
count = 0
for i in range(action_train.shape[0]):
    if action_users[i]:
        action_train_action_users[i] = count
        count += 1

# inverse
inverse_action_train_action_users = dict(map(reversed, action_train_action_users.items()))

adventure_train_action_users = {}
adventure_users = adventure_train.getnnz(1)>0
count = 0
for i in range(adventure_train.shape[0]):
    if adventure_users[i]:
        adventure_train_action_users[i] = count
        count += 1

# inverse
inverse_adventure_train_action_users = dict(map(reversed, adventure_train_action_users.items()))

In [6]:
# それぞれでALSする
# 今回は mediateでやったときのものを使う
action_ALS = cloudpickle.load(open('output/ML-20M-action_ALS.pkl', 'rb'))
adventure_ALS = cloudpickle.load(open("output/ML-20M-adventure_ALS.pkl","rb"))

action_ALS_user_vectors = action_ALS.user_factors
adventure_ALS_user_vectors = adventure_ALS.user_factors

In [7]:
# actionとadventureでoverlapしているユーザで、ベクトルの対応表を作る
overlap_action_user_vectors = []
overlap_adventure_user_vectors = []
count = 0
for u in tqdm(range(X_train.shape[0])):
    if u in action_train_action_users and u in adventure_train_action_users:
        overlap_action_user_vectors.append(action_ALS_user_vectors[action_train_action_users[u]].tolist())
        overlap_adventure_user_vectors.append(adventure_ALS_user_vectors[adventure_train_action_users[u]].tolist())

100%|██████████| 138389/138389 [00:02<00:00, 50574.56it/s]


In [8]:
# AutoEncoderの学習をする
from keras.layers import Input, Dense
from keras.models import Model, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint

def build_model(input_dim, output_dim):
    inputs = Input(shape=(input_dim,))
    encoded = Dense(128, activation='relu')(inputs)
    encoded = Dense(64, activation='relu')(encoded)
    encoded = Dense(32, activation='relu')(encoded)

    decoded = Dense(64, activation='relu')(encoded)
    decoded = Dense(128, activation='relu')(decoded)
    decoded = Dense(output_dim, activation='sigmoid')(decoded)
    autoencoder = Model(inputs, decoded)
    autoencoder.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae','mse'])
    return autoencoder

Using TensorFlow backend.


In [9]:
from sklearn.model_selection import train_test_split
X_train_vector, X_test_vector, y_train_vector, y_test_vector = train_test_split(overlap_action_user_vectors, overlap_adventure_user_vectors, random_state=42)
X_train_vector, X_val_vector, y_train_vector, y_val_vector = train_test_split(X_train_vector, y_train_vector, random_state=42)

epoch_size = 100
batch_size = 256
mcheck = ModelCheckpoint(
    'output/ml-20m-model.h5',
    monitor='val_loss',
    save_best_only=True,
    verbose=1)
es_cb = EarlyStopping(
    monitor='val_loss',
    patience=10,
    verbose=1,
    mode='auto')
model = build_model(np.array(X_train_vector).shape[1], np.array(y_train_vector).shape[1])
model.fit(
    np.array(X_train_vector),
    np.array(y_train_vector),
    batch_size=batch_size,
    epochs=epoch_size,
    validation_data=(
        np.array(X_val_vector),
        np.array(y_val_vector)),
    callbacks=[
        mcheck,
        es_cb],
    shuffle=True,
    verbose=1)


Train on 61820 samples, validate on 20607 samples
Epoch 1/100
61820/61820 [==============================] - 2s 30us/step - loss: 0.3746 - mae: 0.3837 - mse: 0.3746 - val_loss: 0.3447 - val_mae: 0.3566 - val_mse: 0.3447

Epoch 00001: val_loss improved from inf to 0.34468, saving model to output/ml-20m-model.h5
Epoch 2/100
61820/61820 [==============================] - 1s 23us/step - loss: 0.3482 - mae: 0.3567 - mse: 0.3482 - val_loss: 0.3405 - val_mae: 0.3538 - val_mse: 0.3405

Epoch 00002: val_loss improved from 0.34468 to 0.34049, saving model to output/ml-20m-model.h5
Epoch 3/100
61820/61820 [==============================] - 1s 22us/step - loss: 0.3405 - mae: 0.3526 - mse: 0.3405 - val_loss: 0.3277 - val_mae: 0.3480 - val_mse: 0.3277

Epoch 00003: val_loss improved from 0.34049 to 0.32766, saving model to output/ml-20m-model.h5
Epoch 4/100
61820/61820 [==============================] - 1s 22us/step - loss: 0.3197 - mae: 0.3440 - mse: 0.3197 - val_loss: 0.3051 - val_mae: 0.3376 - v

In [10]:
# テストデータに対するRMSE計算
from sklearn.metrics import mean_squared_error
best_model = load_model('output/ml-20m-model.h5')
y_pred = best_model.predict(np.array(X_test_vector))
rmse_ = np.sqrt(mean_squared_error(y_pred, np.array(y_test_vector)))
print('rmse: {}'.format(rmse_))

rmse: 0.5244525355572482


In [11]:
# vaeとで、rmseを比較し、良い方を選ぶ
# ref. https://keras.io/examples/variational_autoencoder/

from keras.layers import Lambda, Input, Dense
from keras.models import Model
from keras.datasets import mnist
from keras.losses import mse, binary_crossentropy
from keras.utils import plot_model
from keras import backend as K

class VAE():
    def __init__(self, input_dim, intermediate_dim, latent_dim, original_dim):
        self.input_dim = input_dim
        self.original_dim = original_dim
        self.intermediate_dim = intermediate_dim
        self.latent_dim = latent_dim
        self.z_mean = None
        self.z_log_var = None


    # reparameterization trick
    # instead of sampling from Q(z|X), sample epsilon = N(0,I)
    # z = z_mean + sqrt(var) * epsilon
    def sampling(self, args):
        """Reparameterization trick by sampling from an isotropic unit Gaussian.

        # Arguments
            args (tensor): mean and log of variance of Q(z|X)

        # Returns
            z (tensor): sampled latent vector
        """

        z_mean, z_log_var = args
        batch = K.shape(z_mean)[0]
        dim = K.int_shape(z_mean)[1]
        # by default, random_normal has mean = 0 and std = 1.0
        epsilon = K.random_normal(shape=(batch, dim))
        return z_mean + K.exp(0.5 * z_log_var) * epsilon


    def vae_loss(self, y_true, y_pred):
        reconstruction_loss = mse(y_true, y_pred)
        reconstruction_loss *= self.original_dim
        kl_loss = 1 + self.z_log_var - K.square(self.z_mean) - K.exp(self.z_log_var)
        kl_loss = K.sum(kl_loss, axis=-1)
        kl_loss *= -0.5
        vae_loss = K.mean(reconstruction_loss + kl_loss)
        return vae_loss


    def build_vae(self):
        # VAE model = encoder + decoder
        # build encoder model
        inputs = Input(shape=(self.input_dim,), name='encoder_input')
        x = Dense(128, activation='relu')(inputs)
        x = Dense(64, activation='relu')(x)
        x = Dense(32, activation='relu')(x)
        self.z_mean = Dense(self.latent_dim, name='z_mean')(x)
        self.z_log_var = Dense(self.latent_dim, name='z_log_var')(x)

        # use reparameterization trick to push the sampling out as input
        # note that "output_shape" isn't necessary with the TensorFlow backend
        z = Lambda(self.sampling, output_shape=(self.latent_dim,), name='z')([self.z_mean, self.z_log_var])

        # instantiate encoder model
        encoder = Model(inputs, [self.z_mean, self.z_log_var, z], name='encoder')

        # build decoder model
        latent_inputs = Input(shape=(self.latent_dim,), name='z_sampling')
        x = Dense(32, activation='relu')(latent_inputs)
        x = Dense(64, activation='relu')(latent_inputs)
        x = Dense(128, activation='relu')(latent_inputs)
        decoder_outputs = Dense(self.original_dim, activation='sigmoid')(x)

        # instantiate decoder model
        decoder = Model(latent_inputs, decoder_outputs, name='decoder')

        # instantiate VAE model
        outputs = decoder(encoder(inputs)[2])
        vae = Model(inputs, outputs, name='vae')
        vae.compile(optimizer='adam', loss=self.vae_loss)
        return vae

In [12]:
vae = VAE(np.array(X_train_vector).shape[1], 256, 2, np.array(y_train_vector).shape[1])
model = vae.build_vae()

In [13]:
model.summary()

Model: "vae"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   (None, 100)               0         
_________________________________________________________________
encoder (Model)              [(None, 2), (None, 2), (N 23396     
_________________________________________________________________
decoder (Model)              (None, 100)               13284     
Total params: 36,680
Trainable params: 36,680
Non-trainable params: 0
_________________________________________________________________


In [14]:
epoch_size = 100
batch_size = 256
mcheck = ModelCheckpoint(
    'output/ml-20m-vae.h5',
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=True,
    verbose=1)
es_cb = EarlyStopping(
    monitor='val_loss',
    patience=10,
    verbose=1,
    mode='auto')
model.fit(
    np.array(X_train_vector),
    np.array(y_train_vector),
    batch_size=batch_size,
    epochs=epoch_size,
    validation_data=(
        np.array(X_val_vector),
        np.array(y_val_vector)),
    callbacks=[
        mcheck,
        es_cb],
    shuffle=True,
    verbose=1)

Train on 61820 samples, validate on 20607 samples
Epoch 1/100
61820/61820 [==============================] - 1s 23us/step - loss: 40.9104 - val_loss: 34.9151

Epoch 00001: val_loss improved from inf to 34.91512, saving model to output/ml-20m-vae.h5
Epoch 2/100
61820/61820 [==============================] - 1s 15us/step - loss: 34.6056 - val_loss: 33.6632

Epoch 00002: val_loss improved from 34.91512 to 33.66316, saving model to output/ml-20m-vae.h5
Epoch 3/100
61820/61820 [==============================] - 1s 15us/step - loss: 33.7799 - val_loss: 33.1645

Epoch 00003: val_loss improved from 33.66316 to 33.16448, saving model to output/ml-20m-vae.h5
Epoch 4/100
61820/61820 [==============================] - 1s 15us/step - loss: 33.5114 - val_loss: 33.0218

Epoch 00004: val_loss improved from 33.16448 to 33.02184, saving model to output/ml-20m-vae.h5
Epoch 5/100
61820/61820 [==============================] - 1s 16us/step - loss: 33.4078 - val_loss: 32.9593

Epoch 00005: val_loss improved

In [15]:
# テストデータに対するRMSE計算
from sklearn.metrics import mean_squared_error
best_model_vae = vae.build_vae()
best_model_vae.load_weights('output/ml-20m-vae.h5')
y_pred = best_model_vae.predict(np.array(X_test_vector))
rmse_ = np.sqrt(mean_squared_error(y_pred, np.array(y_test_vector)))
print('rmse: {}'.format(rmse_))

rmse: 0.5629579415529582


(VAEのやりかたが悪かったようなだけな気もするが)今回は素のautoencoderを採用する

In [16]:
# 評価対象のユーザ
test_adventure_pos_items_dict = {}
for i in tqdm(range(X_test.shape[0])):
    # trainでadventureにアクションしていないユーザに
    rated_items = X_train[i, :].indices
    if len([v for v in rated_items if 'Adventure' in itemid_genres_dict[v]]) == 0:
        # X_testの中でstoreしているアイテムが0以上のユーザに
        if X_test[i, :].nnz > 0:
            test_items = []
            selected_user_ratings = X_test[i, :]
            value_indices = selected_user_ratings.indices
            sorted_indices = np.argsort(-X_test[i, :].toarray())[0]
            # valueがあるアイテムのジャンルがadventureの場合に
            for v in sorted_indices[:len(value_indices)]:
                if 'Adventure' in itemid_genres_dict[v]:
                    test_items.append(v)
            if len(test_items) > 0:
                test_adventure_pos_items_dict[i] = test_items

100%|██████████| 138389/138389 [00:21<00:00, 6388.10it/s]


In [17]:
# adventureのitemのベクトル
adventure_item_vectors = adventure_ALS.item_factors

In [18]:
from lib.recommend_util import ndcg
ndcgs = {
    'ndcg5':  [],
    'ndcg10':  [],
    'ndcg20':  [],
    'ndcg50':  [],
    'ndcg100':  []
}
count = 0
best_model = load_model('output/ml-20m-model.h5')

for userid, pos_items in tqdm(test_adventure_pos_items_dict.items()):
   # pos_itemsをadventure_matrixの次元に変換する
    pos_items = np.array([adventure_concat_itemid_dict[v] for v in pos_items])
    # useridに対応するユーザベクトル(action)を得る
    try:
        action_userid = action_train_action_users[userid]
    except:
        count += 1
        # 推薦できないユーザの場合は無条件で0を入れる
        ndcgs['ndcg5'].append(0)
        ndcgs['ndcg10'].append(0)
        ndcgs['ndcg20'].append(0)
        ndcgs['ndcg50'].append(0)
        ndcgs['ndcg100'].append(0)
        continue
        
    action_user_vector = action_ALS_user_vectors[action_userid, :]
    # autoencoderを使ってadventureの次元に変換する
    adventure_user_vector_action_AE = best_model.predict(action_user_vector.reshape(1, -1))
    # adventureのitemのベクトルと掛け合わせる
    adv_predict = np.dot(adventure_user_vector_action_AE, adventure_item_vectors.T)
    # sum_ratingsをargsort
    sorted_indices = np.array([v for v in np.argsort(-adv_predict)])[0]
    ndcgs['ndcg5'].append(ndcg(sorted_indices[:5], pos_items))
    ndcgs['ndcg10'].append(ndcg(sorted_indices[:10], pos_items))
    ndcgs['ndcg20'].append(ndcg(sorted_indices[:20], pos_items))
    ndcgs['ndcg50'].append(ndcg(sorted_indices[:50], pos_items))
    ndcgs['ndcg100'].append(ndcg(sorted_indices[:100], pos_items))

100%|██████████| 18319/18319 [00:28<00:00, 641.72it/s]


In [19]:
print("ndcg@5: {}".format(np.mean(ndcgs['ndcg5'])))
print("ndcg@10: {}".format(np.mean(ndcgs['ndcg10'])))
print("ndcg@20: {}".format(np.mean(ndcgs['ndcg20'])))
print("ndcg@50: {}".format(np.mean(ndcgs['ndcg50'])))
print("ndcg@100: {}".format(np.mean(ndcgs['ndcg100'])))

ndcg@5: 0.12771399604572783
ndcg@10: 0.166220083945197
ndcg@20: 0.21087733627423522
ndcg@50: 0.280025326182347
ndcg@100: 0.3282249752382082
